## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [23]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [35]:
# 讀取 wine datasets
wine = datasets.load_wine()
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.25, random_state=42)

# 建立模型
clf = RandomForestRegressor()

In [65]:
# 使用預設參數得到的結果 MSE 0.04258
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(metrics.mean_squared_error(y_test, y_pred))

0.042577777777777784


In [28]:
# random cv
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [29]:
random_grid

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
 'max_features': ['auto', 'sqrt'],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'min_samples_split': [2, 5, 10],
 'min_samples_leaf': [1, 2, 4],
 'bootstrap': [True, False]}

In [30]:
clf_random = RandomizedSearchCV(
    estimator = clf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, n_jobs = -1)

# n_jobs = -1，使用所有的CPU進行訓練，預設爲1，使用1個CPU
# verbose: Controls the verbosity: the higher, the more messages. (output message)

In [66]:
clf_random.fit(x_train, y_train)
print(clf_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.4min finished


{'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 40, 'bootstrap': False}


In [68]:
# 使用最佳參數重新建立模型
clf_bestparam = RandomForestRegressor(n_estimators=clf_random.best_params_['n_estimators'],
                                      max_features=clf_random.best_params_['max_features'], 
                                      max_depth = clf_random.best_params_['max_depth'],
                                      min_samples_split = clf_random.best_params_['min_samples_split'],
                                      min_samples_leaf = clf_random.best_params_['min_samples_leaf'],
                                      bootstrap =clf_random.best_params_['bootstrap'], 
                                     )

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

# MSE 些微提升至 0.04502
print(metrics.mean_squared_error(y_test, y_pred))

0.045024688888888886
